### Install alfalfa-client from the development branch

Skip this step if you have done it already.

In [ ]:
%%bash
pip uninstall alfalfa-client
Y

In [ ]:
!pip install git+https://github.com/NREL/alfalfa-client@81541ea52e600dc07f1d8eac553cd0cf14968313

### Import the libraries

In [ ]:
import alfalfa_client.alfalfa_client as ac
import alfalfa_client.historian as ah

import os, requests, datetime

In [ ]:

client = ac.AlfalfaClient()
historian = ah.Historian()


### Submit your testcase. 
Each time you submit a testcase, the function returns its ID. You will need it for issuing calls.

For more information: [Alfalfa's Wiki](https://github.com/NREL/alfalfa/wiki/OpenStudio-Model-Setup)

In [ ]:
zippath = "examples/with_ep2brick.zip"
ref = client.submit(os.path.join('..', zippath))

### Start a testcase
You need its ID, a start and end time. Here, we use an external clock to control the simulation. The external clock can ignore the end time.

In [ ]:

start = datetime.datetime(2022, 1, 1, 0, 0, 0).strftime('%Y-%m-%d %H:%M:%S')
end = datetime.datetime(2023, 1, 1, 0, 0, 0).strftime('%Y-%m-%d %H:%M:%S')

inp = client.start(ref, start_datetime=start, end_datetime=end, external_clock=True)

In [ ]:
client.stop(ref)

### Get testcase information

See all the testcase outputs

In [ ]:
sensors = client.get_outputs(ref)
sensors

See all the testcase inputs

In [ ]:
points = client.get_inputs(ref)
points

### Override controls

Set new inputs

In [ ]:
u = {
    'pvav_pod_3_supply_inlet_node_Air_Flow_Setpoint' : 42.0,  # from the list of points
    #'water_use_connections_1_inlet_water_node_Enthalpy_Setpoint_Enable' : 0.0  # from the list of points
}
client.set_inputs(ref, u)

Advance the simulation by 1-minute timesteps.

In [ ]:
print(f"T0 = {client.get_sim_time(ref)}")
client.advance(ref)
print(f"T1 = {client.get_sim_time(ref)}")

Review the results

In [ ]:
res = client.get_outputs(ref)
res

### Example
We can update the control vector at each timestep. The u vector is a dict.

In [ ]:
steps = 50 # 50 minutes

for t in range(steps):
    command = t % 2
    u = {
    'pvav_pod_3_supply_inlet_node_Air_Flow_Setpoint' : 42.,#command,
    }
    response = client.set_inputs(ref, u)
    client.advance(ref)
    res = client.get_outputs(ref)
    print(f"Sensors: \n{res['pvav_pod_3_supply_inlet_node_Air_Flow_Setpoint_Value']}")
    print(f"Sensors: \n{res['pvav_pod_3_supply_inlet_node_Air_Flow_SensorAir_Flow']}")
    
client.stop(ref)